<a href="https://colab.research.google.com/github/trong-shen/Game-of-Throne-Project/blob/master/GOT_Project_NLP_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the CSV file from Github

In [0]:
import pandas as pd
import numpy as np
import re

GOT= pd.read_csv('https://raw.githubusercontent.com/trong-shen/Game-of-Throne-Project/master/Game_of_Thrones_Script.csv')

#Extract only season one data
GOT1=GOT[GOT.Season=="Season 1"]

print(GOT1.head())
print(GOT1.info())

  Release Date  ...                                           Sentence
0   2011-04-17  ...  What do you expect? They're savages. One lot s...
1   2011-04-17  ...  I've never seen wildlings do a thing like this...
2   2011-04-17  ...                             How close did you get?
3   2011-04-17  ...                            Close as any man would.
4   2011-04-17  ...                   We should head back to the wall.

[5 rows x 6 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3179 entries, 0 to 3178
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Release Date   3179 non-null   object
 1   Season         3179 non-null   object
 2   Episode        3179 non-null   object
 3   Episode Title  3179 non-null   object
 4   Name           3179 non-null   object
 5   Sentence       3179 non-null   object
dtypes: object(6)
memory usage: 173.9+ KB
None


In [0]:
!pip install nltk
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [0]:
GOT1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3179 entries, 0 to 3178
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Release Date   3179 non-null   object
 1   Season         3179 non-null   object
 2   Episode        3179 non-null   object
 3   Episode Title  3179 non-null   object
 4   Name           3179 non-null   object
 5   Sentence       3179 non-null   object
dtypes: object(6)
memory usage: 173.9+ KB


In [0]:
#Data Cleaning and prep for callsification analysis
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#Only take the sentence column (Later for the X)
lines=[];
for i in range(len(GOT1)):
  #Only keep the letter, and replace the removed ones with space.
  sentence = re.sub('[^a-zA-Z]',' ',GOT1['Sentence'][i])
  #Convert all words into lower case.
  sentence = sentence.lower()
  #Split
  sentence = sentence.split()
  #Remove non-relevant words
  ps = PorterStemmer()
  sentence = [ps.stem(word) for word in sentence if not  word in set(stopwords.words('english'))]
  #Join the words together for the Naive Bayes Classification
  sentence = ' '.join(sentence)
  lines.append(sentence)

In [0]:
#create bag od words model -> rows represent each line, each word is one column
from sklearn.feature_extraction.text import CountVectorizer
#Make sure only keep the top 1500 most frequent words for analysis
cv=CountVectorizer(max_features=1500)

In [0]:
#Create X and y -> X should be a large array where (# of records,#of words),y should be the "name" in GOT1
X=cv.fit_transform(lines).toarray()
y=GOT1.iloc[:,4].values

In [0]:
#Make training and test sets for Naive Bays
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [0]:
#Fitting Naive Bays into the training set
from sklearn.naive_bayes import GaussianNB
classifier=GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
#Predicting the test set
y_pred = classifier.predict(X_test)

In [0]:
#Making the confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.08176100628930817